# Setup

In [0]:
import pandas as pd
import numpy as np

In [0]:
ALL_SEASONS = list(range(2013, 2025))
ALL_TEAMS = [
    'ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
    'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LAC', 'LAR', 'LV',
    'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS'
]

In [0]:
def read_df_from_warehouse(table_name):
  return spark.read.table(table_name).toPandas()

def save_table_to_warehouse(df: pd.DataFrame, table_name: str):
  df.to_csv(f'/tmp/{table_name}.csv', index=False)
  spark.createDataFrame(df).write.mode("overwrite").saveAsTable(table_name)

In [0]:
def get_team_code(region_code):
    if region_code == 'LA' or region_code == 'STL':
        return 'LAR'
    if region_code == 'SD':
        return 'LAC'
    if region_code == 'OAK':
        return 'LV'
    return region_code

def get_team_idx(team):
    return ALL_TEAMS.index(team)

def get_outcome_score(home_score, away_score):
    return 1 if home_score > away_score else 0 if home_score < away_score else 0.5

def get_domscores_week(season, week):
    if season <= 2020 and week > 17:
        return 17
    if season > 2020 and week > 18:
        return 18
    return week - 1 


In [0]:
bronze_injuries = read_df_from_warehouse('bronze_injuries')
bronze_snap_counts = read_df_from_warehouse('bronze_snap_counts')
bronze_games = read_df_from_warehouse('bronze_games')
bronze_depth_charts = read_df_from_warehouse('bronze_depth_charts')

# 1. Bronze to Silver

#### a. Weekly Dominance Matrix Scores

In [0]:
silver_domscores = pd.DataFrame(
    columns=['season', 'week', 'team', 'score', 'rank', 'type']
)

In [0]:
# standard dominance matrix
for season in ALL_SEASONS:
  dom_matrix = np.zeros((len(ALL_TEAMS), len(ALL_TEAMS)))

  num_weeks = 18 if season >= 2021 else 17
  for week in range(1, num_weeks + 1):
    # pull weekly games
    games = bronze_games[(bronze_games['season'] == season) & (bronze_games['week'] == week)]

    # update matrix for weekly games
    for i, row in games.iterrows():
      home_team = get_team_code(row['home_team'])
      away_team = get_team_code(row['away_team'])
      home_idx = get_team_idx(home_team)
      away_idx = get_team_idx(away_team)
      if row['home_score'] > row['away_score']:
        dom_matrix[home_idx][away_idx] += 1
      elif row['away_score'] > row['home_score']:
        dom_matrix[away_idx][home_idx] += 1
      elif row['home_score'] == row['away_score']:
        dom_matrix[home_idx][away_idx] += 0.5
        dom_matrix[away_idx][home_idx] += 0.5

    # calculate team scores
    dom_matrix_squared = dom_matrix @ dom_matrix
    sig_w = dom_matrix.sum(axis=1) + dom_matrix_squared.sum(axis=1)

    dom_matrix_transposed = dom_matrix.T
    dom_matrix_transposed_squared = dom_matrix_transposed @ dom_matrix_transposed
    sig_l = dom_matrix_transposed.sum(axis=1) + dom_matrix_transposed_squared.sum(axis=1)

    num_games_played = dom_matrix.sum(axis=1) + dom_matrix_transposed.sum(axis=1)
    score = (sig_w - sig_l) / num_games_played

    # create weekly score dicts
    score_rows = []
    for i, team in enumerate(ALL_TEAMS):
      score_rows.append({
          'season': season,
          'week': week,
          'team': team,
          'score': 0 if score[i] is 'nan' else score[i],
          'type': 'standard'
      })

    score_rows.sort(key=lambda x: x['score'], reverse=True)
    for i, row in enumerate(score_rows):
        row['rank'] = i + 1

    # add score dicts to silver_domscores df
    silver_domscores = pd.concat([silver_domscores, pd.DataFrame(score_rows)])

In [0]:
# point-weighted dominance matrix
for season in ALL_SEASONS:
  dom_matrix = np.zeros((len(ALL_TEAMS), len(ALL_TEAMS)))

  num_weeks = 18 if season >= 2021 else 17
  for week in range(1, num_weeks + 1):
    # pull weekly games
    games = bronze_games[(bronze_games['season'] == season) & (bronze_games['week'] == week)]

    # update matrix for weekly games
    for i, row in games.iterrows():
      home_team = get_team_code(row['home_team'])
      away_team = get_team_code(row['away_team'])
      home_idx = get_team_idx(home_team)
      away_idx = get_team_idx(away_team)
      if row['home_score'] > row['away_score']:
        dom_matrix[home_idx][away_idx] += row['home_score'] - row['away_score']
      elif row['away_score'] > row['home_score']:
        dom_matrix[away_idx][home_idx] += row['away_score'] - row['home_score']
      elif row['home_score'] == row['away_score']:
        dom_matrix[home_idx][away_idx] += 0.5
        dom_matrix[away_idx][home_idx] += 0.5

    # calculate team scores
    dom_matrix_squared = dom_matrix @ dom_matrix
    sig_w = dom_matrix.sum(axis=1) + dom_matrix_squared.sum(axis=1)

    dom_matrix_transposed = dom_matrix.T
    dom_matrix_transposed_squared = dom_matrix_transposed @ dom_matrix_transposed
    sig_l = dom_matrix_transposed.sum(axis=1) + dom_matrix_transposed_squared.sum(axis=1)

    num_games_played = [
        len(
            bronze_games[
                (bronze_games['season'] == season) & 
                (bronze_games['week'] <= week) & 
                ((bronze_games['home_team'].apply(get_team_code) == team) | (bronze_games['away_team'].apply(get_team_code) == team))
            ]
        ) for team in ALL_TEAMS]
    score = (sig_w - sig_l) / num_games_played

    # create weekly score dicts
    score_rows = []
    for i, team in enumerate(ALL_TEAMS):
      score_rows.append({
          'season': season,
          'week': week,
          'team': team,
          'score': 0 if score[i] is 'nan' else score[i],
          'type': 'point-weighted'
      })

    score_rows.sort(key=lambda x: x['score'], reverse=True)
    for i, row in enumerate(score_rows):
        row['rank'] = i + 1

    # add score dicts to silver_domscores df
    silver_domscores = pd.concat([silver_domscores, pd.DataFrame(score_rows)])

In [0]:
# recency-weighted dominance matrix
for season in ALL_SEASONS:
  dom_matrix = np.zeros((len(ALL_TEAMS), len(ALL_TEAMS)))

  num_weeks = 18 if season >= 2021 else 17
  for week in range(1, num_weeks + 1):
    # pull weekly games
    games = bronze_games[(bronze_games['season'] == season) & (bronze_games['week'] == week)]

    # update matrix for weekly games
    for i, row in games.iterrows():
      home_team = get_team_code(row['home_team'])
      away_team = get_team_code(row['away_team'])
      home_idx = get_team_idx(home_team)
      away_idx = get_team_idx(away_team)
      if row['home_score'] > row['away_score']:
        dom_matrix[home_idx][away_idx] += week
      elif row['away_score'] > row['home_score']:
        dom_matrix[away_idx][home_idx] += week
      elif row['home_score'] == row['away_score']:
        dom_matrix[home_idx][away_idx] += 0.5
        dom_matrix[away_idx][home_idx] += 0.5

    # calculate team scores
    dom_matrix_squared = dom_matrix @ dom_matrix
    sig_w = dom_matrix.sum(axis=1) + dom_matrix_squared.sum(axis=1)

    dom_matrix_transposed = dom_matrix.T
    dom_matrix_transposed_squared = dom_matrix_transposed @ dom_matrix_transposed
    sig_l = dom_matrix_transposed.sum(axis=1) + dom_matrix_transposed_squared.sum(axis=1)

    num_games_played = [
        len(
            bronze_games[
                (bronze_games['season'] == season) & 
                (bronze_games['week'] <= week) & 
                ((bronze_games['home_team'].apply(get_team_code) == team) | (bronze_games['away_team'].apply(get_team_code) == team))
            ]
        ) for team in ALL_TEAMS]
    score = (sig_w - sig_l) / num_games_played

    # create weekly score dicts
    score_rows = []
    for i, team in enumerate(ALL_TEAMS):
      score_rows.append({
          'season': season,
          'week': week,
          'team': team,
          'score': 0 if score[i] is 'nan' else score[i],
          'type': 'recency-weighted'
      })

    score_rows.sort(key=lambda x: x['score'], reverse=True)
    for i, row in enumerate(score_rows):
        row['rank'] = i + 1

    # add score dicts to silver_domscores df
    silver_domscores = pd.concat([silver_domscores, pd.DataFrame(score_rows)])

In [0]:
save_table_to_warehouse(silver_domscores, 'silver_domscores')

#### b. Injuries: Silver Edition

In [0]:
silver_injuries = pd.DataFrame(
    columns=['season', 'week', 'player', 'status', 'team', 'snap_pct_when_healthy_o',
             'snap_pct_when_healthy_d', 'snap_pct_when_healthy_st', 'depth_position']
)

In [0]:
for i, row in bronze_injuries.iterrows():
    injury = {
        'season': row['season'],
        'week': row['week'],
        'player': row['full_name'],
        'status': row['report_status'],
        'team': get_team_code(row['team'])
    }

    player_snap_counts = bronze_snap_counts[
        (bronze_snap_counts['season'] == row['season']) & 
        (bronze_snap_counts['week'] < row['week']) & 
        (bronze_snap_counts['player'] == row['full_name'])
    ]

    if len(player_snap_counts):
        injury['snap_pct_when_healthy_o'] = player_snap_counts['offense_pct'].mean()
        injury['snap_pct_when_healthy_d'] = player_snap_counts['defense_pct'].mean()
        injury['snap_pct_when_healthy_st'] = player_snap_counts['st_pct'].mean()
    else:
        injury['snap_pct_when_healthy_o'] = 0
        injury['snap_pct_when_healthy_d'] = 0
        injury['snap_pct_when_healthy_st'] = 0

    player_depth_chart = bronze_depth_charts[
        (bronze_depth_charts['season'] == row['season']) & 
        (bronze_depth_charts['week'] == row['week']) & 
        (bronze_depth_charts['full_name'] == row['full_name'])
    ]

    if len(player_depth_chart):
        injury['depth_position'] = int(player_depth_chart['depth_team'].iloc[0])
    else:
        injury['depth_position'] = 3

    silver_injuries = pd.concat([silver_injuries, pd.DataFrame([injury])])
        
silver_injuries.head()

,season,week,player,status,team,snap_pct_when_healthy_o,snap_pct_when_healthy_d,snap_pct_when_healthy_st,depth_position
0,2021.0,7.0,Azeez Ojulari,None,NYG,0.000000,0.571667,0.073333,2
0,2021.0,7.0,Sam Beal,None,NYG,0.000000,0.000000,0.260000,3
0,2021.0,7.0,Ben Bredeson,None,NYG,0.843333,0.000000,0.000000,1
0,2021.0,7.0,Kaden Smith,None,NYG,0.340000,0.000000,0.296667,2
0,2021.0,7.0,Saquon Barkley,Out,NYG,0.632000,0.000000,0.000000,1


In [0]:
len(silver_injuries)

84684

In [0]:
save_table_to_warehouse(silver_injuries, 'silver_injuries')

# 2. Silver to Gold

In [0]:
bronze_games = read_df_from_warehouse('bronze_games')
silver_injuries = read_df_from_warehouse('silver_injuries')
silver_domscores = read_df_from_warehouse('silver_domscores')

In [0]:
home_team_columns = [
    'home_team_id', 'home_pfpg', 'home_papg', 'home_dom_standard_score', 'home_dom_standard_rank',
    'home_dom_recency_score', 'home_dom_recency_rank', 'home_dom_point_score', 'home_dom_point_rank',
    'home_pct', 'home_rest_days', 'home_injury_impact_snaps_offense', 'home_injury_impact_snaps_defense', 'home_injury_impact_snaps_st', 'home_streak', 'home_starters_out', 'home_starters_questionable', 'home_total_players_out', 'home_total_players_questionable'
]

away_team_columns = [str.replace(col, 'home_', 'away_') for col in home_team_columns]

other_columns = ['home_coach_pct', 'home_win', 'season', 'week', 'coaches_gp']

gold_games = pd.DataFrame(
    columns=home_team_columns + away_team_columns + other_columns
)

In [0]:
bronze_games = bronze_games[bronze_games['season'] >= 2013].copy()
for i, row in bronze_games.iterrows():
    # fields derived directly from current row
    home_team_id = get_team_code(row['home_team'])
    away_team_id = get_team_code(row['away_team'])
    game = {
        'season': row['season'],
        'week': row['week'],
        'home_team_id': home_team_id,
        'away_team_id': away_team_id,
        'home_rest_days': row['home_rest'],
        'away_rest_days': row['away_rest'],
        'home_win': get_outcome_score(row['home_score'], row['away_score'])
    }

    # fields derived from other bronze_games data
    previous_coach_matchups = bronze_games[
        (
            ((bronze_games['season'] == row['season']) & (bronze_games['week'] < row['week'])) |
            (bronze_games['season'] < row['season'])
        ) &
        (
            ((bronze_games['home_coach'] == row['home_coach']) & (bronze_games['away_coach'] == row['away_coach'])) |
            ((bronze_games['home_coach'] == row['away_coach']) & (bronze_games['away_coach'] == row['home_coach']))
        )
    ]

    home_coach_matchup_wins = 0
    home_coach_total_games = len(previous_coach_matchups)
    for j, match in previous_coach_matchups.iterrows():
        if match['home_coach'] == row['home_coach']:
            home_coach_matchup_wins += get_outcome_score(match['home_score'], match['away_score'])
        else:
            home_coach_matchup_wins += get_outcome_score(match['away_score'], match['home_score'])

    game['home_coach_pct'] = home_coach_matchup_wins / home_coach_total_games if home_coach_total_games > 0 else 0.5
    game['coaches_gp'] = home_coach_total_games

    home_team_previous_home_games = bronze_games[
        (bronze_games['season'] == row['season']) & 
        (bronze_games['week'] < row['week']) & 
        (bronze_games['home_team'] == row['home_team'])
    ].copy()

    home_team_previous_home_games.loc[:, 'pf'] = home_team_previous_home_games['home_score']
    home_team_previous_home_games.loc[:, 'pa'] = home_team_previous_home_games['away_score']
    
    home_team_previous_away_games = bronze_games[
        (bronze_games['season'] == row['season']) & 
        (bronze_games['week'] < row['week']) & 
        (bronze_games['away_team'] == row['home_team'])
    ].copy()

    home_team_previous_away_games.loc[:, 'pf'] = home_team_previous_away_games['away_score']
    home_team_previous_away_games.loc[:, 'pa'] = home_team_previous_away_games['home_score']
    
    home_team_all_previous_games = pd.concat([
        home_team_previous_home_games[['pf', 'pa', 'week']], home_team_previous_away_games[['pf', 'pa', 'week']]
    ]).copy()

    home_team_all_previous_games.loc[:, 'win'] = home_team_all_previous_games.apply(
        lambda x: get_outcome_score(x['pf'], x['pa']), axis=1
    )

    game['home_pfpg'] = home_team_all_previous_games['pf'].mean()
    game['home_papg'] = home_team_all_previous_games['pa'].mean()
    game['home_pct'] = home_team_all_previous_games['win'].mean()

    home_result_series = home_team_all_previous_games.sort_values(by='week')['win']

    home_team_streak = 0
    for j, result in enumerate(home_result_series):
        if result != home_result_series.iloc[j]:
            break
        if result == 1:
            home_team_streak += 1
        if result == 0:
            home_team_streak -= 1
    
    game['home_streak'] = home_team_streak

    away_team_previous_home_games = bronze_games[
        (bronze_games['season'] == row['season']) & 
        (bronze_games['week'] < row['week']) & 
        (bronze_games['home_team'] == row['away_team'])
    ].copy()

    away_team_previous_home_games.loc[:, 'pf'] = away_team_previous_home_games['home_score']
    away_team_previous_home_games.loc[:, 'pa'] = away_team_previous_home_games['away_score']
    
    away_team_previous_away_games = bronze_games[
        (bronze_games['season'] == row['season']) & 
        (bronze_games['week'] < row['week']) & 
        (bronze_games['away_team'] == row['away_team'])
    ].copy()

    away_team_previous_away_games.loc[:, 'pf'] = away_team_previous_away_games['away_score']
    away_team_previous_away_games.loc[:, 'pa'] = away_team_previous_away_games['home_score']
    
    away_team_all_previous_games = pd.concat([
        away_team_previous_home_games[['pf', 'pa', 'week']], away_team_previous_away_games[['pf', 'pa', 'week']]
    ]).copy()

    away_team_all_previous_games.loc[:, 'win'] = away_team_all_previous_games.apply(
        lambda x: get_outcome_score(x['pf'], x['pa']), axis=1
    )

    game['away_pfpg'] = away_team_all_previous_games['pf'].mean()
    game['away_papg'] = away_team_all_previous_games['pa'].mean()
    game['away_pct'] = away_team_all_previous_games['win'].mean()

    away_result_series = away_team_all_previous_games.sort_values(by='week')['win']

    away_team_streak = 0
    for j, result in enumerate(away_result_series):
        if result != away_result_series.iloc[j]:
            break
        if result == 1:
            away_team_streak += 1
        if result == 0:
            away_team_streak -= 1
    
    game['away_streak'] = away_team_streak

    # fields derived from dominance matrix table
    dom_score_week = get_domscores_week(row['season'], row['week'])

    home_team_standard_matrix_df = silver_domscores[
        (silver_domscores['season'] == row['season']) & 
        (silver_domscores['team'] == home_team_id) &
        (silver_domscores['week'] == dom_score_week) &
        (silver_domscores['type'] == 'standard')
    ]
    home_team_points_matrix_df = silver_domscores[
        (silver_domscores['season'] == row['season']) & 
        (silver_domscores['team'] == home_team_id) &
        (silver_domscores['week'] == dom_score_week) &
        (silver_domscores['type'] == 'point-weighted')
    ]
    home_team_recency_matrix_df = silver_domscores[
        (silver_domscores['season'] == row['season']) & 
        (silver_domscores['team'] == home_team_id) &
        (silver_domscores['week'] == dom_score_week) &
        (silver_domscores['type'] == 'recency-weighted')
    ]
    away_team_standard_matrix_df = silver_domscores[
        (silver_domscores['season'] == row['season']) & 
        (silver_domscores['team'] == away_team_id) &
        (silver_domscores['week'] == dom_score_week) &
        (silver_domscores['type'] == 'standard')
    ]
    away_team_points_matrix_df = silver_domscores[
        (silver_domscores['season'] == row['season']) & 
        (silver_domscores['team'] == away_team_id) &
        (silver_domscores['week'] == dom_score_week) &
        (silver_domscores['type'] == 'point-weighted')
    ]
    away_team_recency_matrix_df = silver_domscores[
        (silver_domscores['season'] == row['season']) & 
        (silver_domscores['team'] == away_team_id) &
        (silver_domscores['week'] == dom_score_week) &
        (silver_domscores['type'] == 'recency-weighted')
    ]

    if dom_score_week <= 1:
        game['home_dom_standard_score'] = 0
        game['home_dom_point_score'] = 0
        game['home_dom_recency_score'] = 0
        game['home_dom_standard_rank'] = 0
        game['home_dom_point_rank'] = 0
        game['home_dom_recency_rank'] = 0
        game['away_dom_standard_score'] = 0
        game['away_dom_point_score'] = 0
        game['away_dom_recency_score'] = 0
        game['away_dom_standard_rank'] = 0
        game['away_dom_point_rank'] = 0
        game['away_dom_recency_rank'] = 0
    else:
        try:
            game['home_dom_standard_score'] = home_team_standard_matrix_df['score'].iloc[0]
        except IndexError:
            raise IndexError(
                f'iloc out of bounds, dom_score_week={dom_score_week}, season={row['season']}, home_team={home_team_id}, week={dom_score_week}'
            )
        game['home_dom_point_score'] = home_team_points_matrix_df['score'].iloc[0]
        game['home_dom_recency_score'] = home_team_recency_matrix_df['score'].iloc[0]
        game['home_dom_standard_rank'] = home_team_standard_matrix_df['rank'].iloc[0]
        game['home_dom_point_rank'] = home_team_points_matrix_df['rank'].iloc[0]
        game['home_dom_recency_rank'] = home_team_recency_matrix_df['rank'].iloc[0]
        game['away_dom_standard_score'] = away_team_standard_matrix_df['score'].iloc[0]
        game['away_dom_point_score'] = away_team_points_matrix_df['score'].iloc[0]
        game['away_dom_recency_score'] = away_team_recency_matrix_df['score'].iloc[0]
        game['away_dom_standard_rank'] = away_team_standard_matrix_df['rank'].iloc[0]
        game['away_dom_point_rank'] = away_team_points_matrix_df['rank'].iloc[0]
        game['away_dom_recency_rank'] = away_team_recency_matrix_df['rank'].iloc[0]

    # fields derived from injuries
    home_team_injuries = silver_injuries[
        (silver_injuries['season'] == row['season']) & 
        (silver_injuries['team'] == home_team_id) &
        (silver_injuries['week'] == row['week']) &
        (silver_injuries['status'] != None)
    ]
    
    game['home_injury_impact_snaps_offense'] = home_team_injuries['snap_pct_when_healthy_o'].sum()
    game['home_injury_impact_snaps_defense'] = home_team_injuries['snap_pct_when_healthy_d'].sum()
    game['home_injury_impact_snaps_st'] = home_team_injuries['snap_pct_when_healthy_st'].sum()
    game['home_starters_out'] = len(
        home_team_injuries[
            (home_team_injuries['status'] == 'Out') &
            (home_team_injuries['depth_position'] == 1)
        ]
    )
    game['home_starters_questionable'] = len(
        home_team_injuries[
            (home_team_injuries['status'] != 'Out') &
            (home_team_injuries['status'] != None) &
            (home_team_injuries['depth_position'] == 1)
        ]
    )
    game['home_total_players_out'] = len(
        home_team_injuries[
            (home_team_injuries['status'] == 'Out')
        ]
    )
    game['home_total_players_questionable'] = len(
        home_team_injuries[
            (home_team_injuries['status'] != 'Out') &
            (home_team_injuries['status'] != None)
        ]
    )

    away_team_injuries = silver_injuries[
        (silver_injuries['season'] == row['season']) & 
        (silver_injuries['team'] == away_team_id) &
        (silver_injuries['week'] == row['week']) &
        (silver_injuries['status'] != None)
    ]
    
    game['away_injury_impact_snaps_offense'] = away_team_injuries['snap_pct_when_healthy_o'].sum()
    game['away_injury_impact_snaps_defense'] = away_team_injuries['snap_pct_when_healthy_d'].sum()
    game['away_injury_impact_snaps_st'] = away_team_injuries['snap_pct_when_healthy_st'].sum()
    game['away_starters_out'] = len(
        away_team_injuries[
            (away_team_injuries['status'] == 'Out') &
            (away_team_injuries['depth_position'] == 1)
        ]
    )
    game['away_starters_questionable'] = len(
        away_team_injuries[
            (away_team_injuries['status'] != 'Out') &
            (away_team_injuries['status'] != None) &
            (away_team_injuries['depth_position'] == 1)
        ]
    )
    game['away_total_players_out'] = len(
        away_team_injuries[
            (away_team_injuries['status'] == 'Out')
        ]
    )
    game['away_total_players_questionable'] = len(
        away_team_injuries[
            (away_team_injuries['status'] != 'Out') &
            (away_team_injuries['status'] != None)
        ]
    )

    gold_games = pd.concat([gold_games, pd.DataFrame([game])])

gold_games.head()

,home_team_id,home_pfpg,home_papg,home_dom_standard_score,home_dom_standard_rank,home_dom_recency_score,home_dom_recency_rank,home_dom_point_score,home_dom_point_rank,home_pct,home_rest_days,home_injury_impact_snaps_offense,home_injury_impact_snaps_defense,home_injury_impact_snaps_st,home_streak,home_starters_out,home_starters_questionable,home_total_players_out,home_total_players_questionable,away_team_id,away_pfpg,away_papg,away_dom_standard_score,away_dom_standard_rank,away_dom_recency_score,away_dom_recency_rank,away_dom_point_score,away_dom_point_rank,away_pct,away_rest_days,away_injury_impact_snaps_offense,away_injury_impact_snaps_defense,away_injury_impact_snaps_st,away_streak,away_starters_out,away_starters_questionable,away_total_players_out,away_total_players_questionable,home_coach_pct,home_win,season,week,coaches_gp
0,DAL,20.583333,18.583333,1.166667,10,121.750000,7,80.166667,12,0.583333,10,5.425056,2.628798,2.328242,2,2,5,4,8,PHI,21.500000,22.166667,-0.291667,16,17.458333,13,29.750000,15,0.500000,6,1.097833,3.350000,1.119286,0,2,4,2,5,0.6,1,2018,14,5
0,LV,18.333333,30.583333,-3.958333,31,-143.333333,30,-597.708333,31,0.166667,7,4.691288,2.025758,1.608283,-8,0,9,0,11,PIT,28.833333,23.500000,1.750000,7,68.958333,10,303.958333,5,0.625000,7,5.061667,1.740195,2.672903,3,2,6,2,9,0.5,1,2018,14,0
0,CHI,28.666667,20.083333,1.333333,8,60.000000,11,419.666667,4,0.666667,7,1.033500,1.444273,2.195485,4,0,2,0,6,LAR,34.916667,24.833333,5.750000,1,230.666667,1,432.375000,3,0.916667,7,2.117500,0.890833,0.565000,10,0,3,1,3,0.5,1,2018,14,0
0,SEA,26.583333,21.583333,1.291667,9,85.208333,9,178.791667,10,0.583333,8,3.515338,1.804167,2.139260,2,1,6,1,10,MIN,22.916667,22.500000,0.333333,14,12.416667,15,-167.708333,24,0.541667,8,2.538902,2.823636,1.704053,1,1,4,3,6,1.0,1,2018,14,2
0,KC,36.230769,27.000000,4.846154,2,186.423077,5,449.038462,3,0.846154,4,3.821385,1.828205,0.620385,9,1,7,1,7,LAC,28.153846,20.769231,3.461538,4,200.192308,4,272.423077,6,0.769231,4,1.389013,1.212222,0.956103,7,0,1,3,2,1.0,0,2018,15,3


In [0]:
save_table_to_warehouse(gold_games, 'gold_games')

In [0]:
len(gold_games)

3277

In [0]:
gold_games['season'].value_counts()

2021    285
2023    285
2024    285
2022    284
2020    269
2018    267
2019    267
2015    267
2016    267
2017    267
2013    267
2014    267
Name: season, dtype: int64